## Notebook that trains a Wasserstein-GAN model for galaxy generation

Adapted from: https://github.com/eriklindernoren/PyTorch-GAN/tree/master/implementations/wgan


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image
import torchvision
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
import torchvision.utils as vutils
import matplotlib.animation as animation
from IPython.display import HTML

# set up accordingly
data_dir = "../../data" # directory with data files
labeled_image_dir = "labeled"       # folder within data directory with labeled images (0, 1)
scored_image_dir = "scored_128"      # folder within data directory with scored images
device = torch.device("cuda")        # cuda or cpu
collab = False                        # google collab flag

In [2]:
# google collab
# mount drive, copy over data as zip and unzip it
if collab:
  from google.colab import drive
  drive.mount('/content/drive')
  collab_dir = "content"

  zip_path = os.path.join(data_dir, 'labeled.zip')
  !cp '{zip_path}' .
  !unzip -q labeled.zip
  !rm labeled.zip

  zip_path = os.path.join(data_dir, 'scored_128.zip')
  !cp '{zip_path}' .
  !unzip -q scored_128.zip
  !rm scored_128.zip
else:
    labeled_image_dir = os.path.join(data_dir, labeled_image_dir)
    scored_image_dir = os.path.join(data_dir, scored_image_dir)

In [3]:
class GalaxyDataset(torch.utils.data.Dataset):
    """
    Galaxy dataset class
    Builds a dataset from the labeled and scored images. 
    Requires a threshold score for scored images. 
    Images with a score below the threshold are not used.
    """

    def __init__(self, csv_file, image_dir, scored_dir=None, scores_file=None, transform=None, train=True, size=(128, 128), train_split=0.8, scored_threshold=0):
        self.labels = pd.read_csv(csv_file, index_col="Id")
        self.labels = self.labels[self.labels['Actual'] == 1.0]
        self.size = size
        self.scores = None
        if scores_file is not None:
          self.scores = pd.read_csv(scores_file, index_col="Id")
        self.samples = []
        if train == True:
          self.labels = self.labels[:int(self.labels.shape[0]*train_split)]
        else:
          self.labels = self.labels[int(self.labels.shape[0]*train_split):]
        self.image_dir = image_dir
        self.transform = transform
        self.scored_dir = scored_dir
        self.scored_threshold = scored_threshold
        self.load_dataset()
    def __len__(self):
        return len(self.samples)

    def load_dataset(self):
      print("Loading Dataset...")
      for id, _ in self.labels.iterrows():
        img_name = os.path.join(self.image_dir,
                                  str(id)+'.png')
        self.samples.append(Image.open(img_name).resize(self.size))
      
      if self.scores is not None:
        for id, score in self.scores.iterrows():
          if score.item() > self.scored_threshold:

            img_name = os.path.join(self.scored_dir,
                                      str(id)+'.png')
            self.samples.append(Image.open(img_name).resize(self.size))
        
      print("Dataset Loaded")

    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.samples[idx]
        if self.transform:
            image = self.transform(image)

        return image

Load Dataset with appropriate transforms


In [4]:
batch_size = 64
size = (64, 64)
latent_dim = 64
train_transformation = torchvision.transforms.Compose([
                            torchvision.transforms.RandomAffine(30, translate=(0.2, 0.2)),
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(0, 255.0),
                            torchvision.transforms.Normalize(0.5, 0.5) ## second normalization for tanh
])
val_transformation = torchvision.transforms.Compose([
                            torchvision.transforms.RandomAffine(30, translate=(0.2, 0.2)),
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(0, 255.0),
                            torchvision.transforms.Normalize(0.5, 0.5) ## second normalization for tanh
])


train_dataset = GalaxyDataset(os.path.join(data_dir, "labeled.csv"), labeled_image_dir, scored_dir=scored_image_dir, scores_file=os.path.join(data_dir, "scored.csv"), transform=train_transformation, train=True, size=size, train_split=1.0, scored_threshold=2.60)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=1, pin_memory=True)
print("{} images loaded".format(len(train_dataset)))

Loading Dataset...
Dataset Loaded
2754 images loaded


Set up initialization of weights, filters of generator and discriminator

In [0]:
gen_base_filters = 64
disc_base_filters = 64

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [0]:

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( latent_dim, gen_base_filters * 8, 4, 1, 0, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters * 8),
            # state size. (gen_base_filters*8) x 4 x 4
            nn.ConvTranspose2d(gen_base_filters * 8, gen_base_filters * 4, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters * 4),
            # state size. (gen_base_filters*4) x 8 x 8
            nn.ConvTranspose2d( gen_base_filters * 4, gen_base_filters * 2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters * 2),
            # state size. (gen_base_filters*2) x 16 x 16
            nn.ConvTranspose2d( gen_base_filters * 2, gen_base_filters, 4, 2, 1, bias=False),
            nn.ReLU(True),
            nn.BatchNorm2d(gen_base_filters),
            # state size. (gen_base_filters) x 32 x 32
            nn.ConvTranspose2d( gen_base_filters, 1, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [9]:
gen = Generator().to(device)
gen.apply(weights_init)
print(gen)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(64, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): ReLU(inplace=True)
    (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): ReLU(inplace=True)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): ReLU(inplace=True)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [0]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(1, disc_base_filters, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (disc_base_filters) x 32 x 32
            nn.Conv2d(disc_base_filters, disc_base_filters * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(disc_base_filters * 2),
            # state size. (disc_base_filters*2) x 16 x 16
            nn.Conv2d(disc_base_filters * 2, disc_base_filters * 4, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(disc_base_filters * 4),
            # state size. (disc_base_filters*4) x 8 x 8
            nn.Conv2d(disc_base_filters * 4, disc_base_filters * 8, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm2d(disc_base_filters * 8),
            # state size. (disc_base_filters*8) x 4 x 4
            nn.Conv2d(disc_base_filters * 8, 1, 4, 1, 0, bias=False)
        )

    def forward(self, input):
        return self.main(input)

In [11]:
disc = Discriminator().to(device)
disc.apply(weights_init)
print(disc)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): LeakyReLU(negative_slope=0.2, inplace=True)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): LeakyReLU(negative_slope=0.2, inplace=True)
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)


Set up training parameters

In [0]:
G_losses = []
D_losses = []
img_list = []

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

gen_lr = 0.00005
disc_lr = 0.00005
epochs = 2000
clip_value = 0.01
n_critic = 5
# Setup RMSprop optimizers for both G and D
optimizerD = optim.RMSprop(disc.parameters(), lr=disc_lr)
optimizerG = optim.RMSprop(gen.parameters(), lr=gen_lr)

Function that takes one WGAN train step



In [0]:
def wgan_step(batch_num, batch, disc, gen):
  ############################
  # (1) Update D network
  ###########################
  disc.zero_grad()
  # Format batch
  real = data.to(device)
  b_size = real.size(0)
  # Generate batch of latent vectors
  noise = torch.randn(b_size, latent_dim, 1, 1, device=device)
  # Generate fake image batch with G
  fake = gen(noise)
  loss_D = -torch.mean(disc(real)) + torch.mean(disc(fake))
  loss_D.backward()
 

  ############################
  # (2) Update G network
  ###########################
  # Train the generator every n_critic iterations
  if batch_num % n_critic == 0:

    gen.zero_grad()

    # Generate a batch of images
    gen_imgs = gen(noise)
    # Adversarial loss
    loss_G = -torch.mean(disc(gen_imgs))

    loss_G.backward()

    # Output training stats
    print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
          % (epoch, epochs, i, len(train_loader),
              loss_D.item(), loss_G.item()))
    return loss_D, loss_G
  return loss_D, None

Train and periodically save outputs from generator

In [0]:
# save a grid of images generated from the same initial noise
# also save those same images but changed so that the smallest value
# corresponds to 0, and the largest to 1 (normalized)
# this allows us to see some patterns that would otherwise
# just appear as black
results_dir = "results_wgan"
if not os.path.exists(os.path.join(outf, results_dir)):
  os.mkdir(os.path.join(outf, results_dir))
  os.mkdir(os.path.join(outf, results_dir, "normalized"))
  os.mkdir(os.path.join(outf, results_dir, "unnormalized"))
for epoch in range(1, epochs + 1):
    # For each batch in the dataloader
    for i, data in enumerate(train_loader, 0):

        errD, errG = wgan_step(i, data, disc, gen)
        
        if errG is not None:
          # Update G
          optimizerG.step()
          # Save Losses for plotting later
          G_losses.append(errG.item())
        # Update D
        optimizerD.step()

        # clip weights
        for p in disc.parameters():
            p.data.clamp_(-clip_value, clip_value)
        # Save Losses for plotting later
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (epoch % 5 == 0 and i == len(train_loader)-1):
            with torch.no_grad():
                fake = gen(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, pad_value=255, normalize=True, range=(-1, 1)))
            # also save normalized so we can see patterns / mode collapse
            vutils.save_image(fake,  os.path.join(outf, results_dir, "normalized") + "/epoch" + str(epoch) + ".png", padding=2, pad_value=1, normalize=True)
            vutils.save_image(fake, os.path.join(outf, results_dir, "unnormalized") + "/epoch" + str(epoch) + ".png", padding=2, pad_value=1, normalize=True, range=(-1, 1))

In [0]:
torch.save(gen.state_dict(), os.path.join(outf, results_dir, "wgen.model"))
torch.save(disc.state_dict(), os.path.join(outf, results_dir, "wdisc.model"))
if collab:
  !cp '{outf}'/'{results_dir}'wgen.model '{data_dir}'/'{results_dir}'
  !cp '{outf}'/'{results_dir}' wdisc.model '{data_dir}'/'{results_dir}'
  !cp -rf /'{outf}'/'{results_dir}' '{data_dir}'

Plot train loss

In [0]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

Check how images progressed

In [0]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())